In [ ]:
from __future__ import absolute_import, division
from BashColors import C
from CV2_Utils_2 import *
from time import perf_counter
import os
from os.path import *
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'

contentPath=os.getcwd()
genPath=join(contentPath, 'DataGenerator')
testPath=join(contentPath, 'images')

import tflite_model_maker
from tflite_model_maker import image_classifier, model_spec
from tflite_model_maker.image_classifier import DataLoader
print(tflite_model_maker.__version__)

In [ ]:
# Load input data specific to an on-device ML app.
data = DataLoader.from_folder(genPath)
train_data, restData = data.split(0.9)
print(train_data.size//32)
validation_data, test_data = restData.split(0.5)

In [ ]:
# Load input data specific to an on-device ML app.
testData = DataLoader.from_folder(testPath)
print(testData.index_to_label)
# validation_data, test_data = restData.split(0.5)

In [ ]:
# help(image_classifier.create)

In [ ]:
start=perf_counter()
# Customize the TensorFlow model.
model = image_classifier.create(
    train_data,
    model_spec=model_spec.get('mobilenet_v2'),
    epochs=1,
    dropout_rate=0.25,
    learning_rate=0.001,
    momentum=0.9)

finish=perf_counter()
cvu.printTime(start, finish)

In [ ]:
# accuracy: 90.0%
# Evaluate the model.
loss, accuracy = model.evaluate(testData)

loss = round(loss, 4)
accuracy = round(accuracy, 4)
print(f'loss: {loss}\taccuracy: {accuracy * 100}%')


In [ ]:
length=len(test_ds)
def get_label_color(val1, val2):
  if val1 == val2:
    return 'green'
  else:
    return 'red'
num=len(test_ds)
plt.figure(figsize=(224, 224))
plt.rcParams.update({'font.size': 10})
predicts = model.predict_top_k(test_ds)
for i, (image, label) in enumerate(test_ds.gen_dataset().unbatch().take(num)):
    ax = plt.subplot(num, 1, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(image.numpy(), cmap=plt.cm.gray) # .gray

    predict_label = predicts[i][0][0]
    color = get_label_color(predict_label,
                          test_ds.index_to_label[label.numpy()])
    ax.xaxis.label.set_color(color)
    plt.xlabel('Predicted: %s' % predict_label)
plt.show()